In [1]:
# Import dependencies.
from citipy import citipy
import numpy as np
import pandas as pd
import random
import requests as req
import json
# Import Open Weather Map API keys.
from openWeatherMapApiKeys import apiKey

In [2]:
# Declare variables describing the scope of lat/lng search for cities.
latDim = {'min': -90, 'max': 90}
lngDim = {'min': -180, 'max': 180}

latVals = np.arange(latDim['min'], latDim['max'], 0.1)
lngVals = np.arange(lngDim['min'], lngDim['max'], 0.1)

In [3]:
# Create an empty data frame that will contain details of our randomly selected cities.
colNames = ('cityName', 'countryCode', 'randLat', 'randLng', 'uniqueName')
cities = pd.DataFrame(columns=(colNames))

In [4]:
# Query Citipy for random lat long values until we collect our sample.

sampleSize = 5

counter = 0
while counter < sampleSize:
    randLat = random.choice(latVals)
    randLng = random.choice(lngVals)
    city = citipy.nearest_city(randLat, randLng)
    if city.city_name + city.country_code in cities.uniqueName.tolist():
        pass
    else:
        cities.loc[len(cities)] = [city.city_name, city.country_code, randLat, randLng, city.city_name + city.country_code]
    counter += 1


In [5]:
cities.head()

,cityName,countryCode,randLat,randLng,uniqueName
0,yamada,jp,38.4,145.7,yamadajp
1,sangar,ru,63.7,125.9,sangarru
2,cherskiy,ru,79.8,163.4,cherskiyru
3,san patricio,mx,0.8,-116.6,san patriciomx
4,amapa,br,2.1,-48.7,amapabr


In [ ]:
len(cities)

In [11]:
# Extend data frame to capture weather data from the Open Weather Map.

# cities['lat'] = ''
# cities['lng'] = ''
# cities['Temperature'] = ''
# cities['Humidity'] = ''
# cities['Cloudiness'] = ''
# cities['WindSpeed'] = ''

# HTTP format for this OWM call: 
#'http://api.openweathermap.org/data/2.5/weather?q=luderitzna,na&units=imperial&APPID=key'
baseUrl = 'http://api.openweathermap.org/data/2.5/weather?q='
units = 'imperial'
for index, row in cities.iterrows():
    url = baseUrl + row["cityName"] + ',' + cities["countryCode"] + '&units=' + units + '&APPID=' + apiKey
    weather_response = req.get(url[0])
    weather_json = weather_response.json()
    cities.set_value(index, "Latitude", weather_json["coord"]["lat"])
    cities.set_value(index, "Longitude", weather_json["coord"]["lon"])
    cities.set_value(index, "Temperature", weather_json["main"]["temp"])
    cities.set_value(index, "Humidity", weather_json["main"]["humidity"])
    cities.set_value(index, "Cloudiness", weather_json["clouds"]["all"])
    cities.set_value(index, "WindSpeed", weather_json["wind"]["speed"])

 

yamada
sangar
cherskiy
san patricio
amapa


In [12]:
print(cities)

       cityName countryCode  randLat  randLng      uniqueName  Latitude  \
0        yamada          jp     38.4    145.7        yamadajp     39.47   
1        sangar          ru     63.7    125.9        sangarru     60.29   
2      cherskiy          ru     79.8    163.4      cherskiyru     68.75   
3  san patricio          mx      0.8   -116.6  san patriciomx     28.02   
4         amapa          br      2.1    -48.7         amapabr      0.04   

   Longitude  Temperature  Humidity  Cloudiness  WindSpeed  
0     141.95        73.18      90.0        48.0       9.86  
1      25.04        51.80      93.0        88.0       1.12  
2     161.30        47.08      68.0        64.0       2.82  
3     -97.52        88.72      66.0        20.0      16.11  
4     -51.07        82.40      78.0        20.0       8.05  
